In [ ]:
import sirf.STIR as STIR

In [ ]:
def crop_image(templ_sino, image, nx, ny, nz, slice = None):
    """Crop from (vol_z,vol_y,vol_x) to (nz,ny,nx) and save to file"""
    vol = image.as_array()
    vol_dims = vol.shape
    x_origin = vol_dims[2]//2
    y_origin = vol_dims[1]//2
    if slice is None:
        z_origin = vol_dims[0]//2
    else:
        z_origin = slice
    
    vol = vol[z_origin-nz//2:z_origin+nz//2+nz%2,y_origin-ny//2:y_origin+ny//
              2+ny%2,x_origin-nx//2:x_origin+nx//2+nx%2]
    im = STIR.ImageData(templ_sino)
    dim=(nz,ny,nx)
    vol = vol.reshape(dim)
    voxel_size=im.voxel_sizes()
    im.initialise(dim,voxel_size)
    im.fill(vol)
    return im

In [ ]:
def create_sample_image(image):
    '''fill the image with some simple geometric shapes.'''
    im_shape = image.shape
    image.fill(0)
    
    # create a body-like ellipsoid shape
    shape = STIR.EllipticCylinder()
    shape.set_length(400)
    shape.set_radii((im_shape[1]//3*4, im_shape[2]//2*3))
    shape.set_origin((0, 0, 0))

    # add the shape to the image
    image.add_shape(shape, scale = 1)

    # add some lung-like shapes shape
    shape.set_radii((im_shape[1], im_shape[2]//2))
    shape.set_origin((0,0, -im_shape[2]//3*2))
    image.add_shape(shape, scale = -0.5)

    shape.set_origin((0, 0, im_shape[2]//3*2))
    image.add_shape(shape, scale = -0.5)

    # a spine-like shape
    shape.set_radii((im_shape[1]//4, im_shape[2]//4))
    shape.set_origin((0, im_shape[1]//5*4, 0))
    image.add_shape(shape, scale = 0.5)

    # and a lung tumour-like shape
    shape.set_radii((im_shape[1]//5, im_shape[2]//5))
    shape.set_origin((0, -im_shape[1]//5, im_shape[2]//2))
    image.add_shape(shape, scale = 0.75)
    
    return image

In [ ]:
templ_sino = STIR.AcquisitionData("./template_sinogram.hs")
im = create_sample_image(templ_sino.create_uniform_image(0))
im = crop_image(templ_sino, im, templ_sino.dimensions()[3], templ_sino.dimensions()[2], templ_sino.dimensions()[1])
im = im.zoom_image((0.5,1,1))
att_STIR = im/10 # approximate attenuation coefficient image used in STIR (/cm)
att_SIMIND = att_STIR/0.15*1000 # approximate density image used in SIMIND (mg/cm^3)

In [ ]:
im.show()

In [ ]:
im.write("./image.smi")
att_STIR.write("./attenuation_stir.hv")
att_SIMIND.write("./attenuation_simind.dmi")

In [ ]:
acq_model_matrix = STIR.SPECTUBMatrix()
acq_model_matrix.set_resolution_model(0.2,0.2,full_3D=False)

In [ ]:
im.show()

In [ ]:
%%bash
simind input.smc output/NN:0.001/PX:0.4/FS:image/FD:attenuation_simind

In [ ]:
%%bash
sh ./convertSIMINDToSTIR.sh output.h00

In [ ]:
simulated_data = STIR.AcquisitionData("./output.hs")
simulated_data.show()

In [ ]:
am = STIR.AcquisitionModelUsingMatrix(acq_model_matrix)
am.set_up(simulated_data, im)

In [ ]:
res = am.backward(simulated_data)

In [ ]:
res.show()